In [5]:
# Not needed if pysleep is pip installed (i.e. you probably dont need these lines)
import sys, os
file_dir = os.path.abspath('')
sys.path.insert(0, file_dir+'/../')

In [6]:
#%% Import the tools we need
from mednickdb_pysleep import process_sleep_record, scorefiles, sleep_architecture
import pandas as pd
import numpy as np
import yaml
import mne
import warnings
import datetime
warnings.filterwarnings("ignore")

In [7]:
#setup location of files and save locations
edf_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/'
scorefile_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/sleep_scoring/'
edf_filenames = ['Sid1_edf.edf']
scorefile_filenames = ['sid1.csv']
ids = ['1'] #these are the ids for each subject, they conrespond to each edf/scorefile above
study_settings_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/choc_type_study_settings.yaml'
save_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/'
nighttime_split_method = 'quartiles' #None or quartiles
spindle_algo = 'Wamsley2012'
do_artifacting = False
do_band_power=False
do_spindles=True
do_slow_osc=True
do_overlap=False

In [12]:
#%% loop through edf's, extract band power, spindles and slow osc (per stage)
edf_filepaths = [edf_base_path + edf_filename for edf_filename in edf_filenames]
scoring_filepaths = [scorefile_base_path + scorefile_filename for scorefile_filename in scorefile_filenames]
features_averages_cont = []
band_power_averages_cont = []
features_df_cont = []

study_settings = yaml.safe_load(open(study_settings_path,'r+'))

for edf_filepath, scorefile_filepath, id_ in zip(edf_filepaths, scoring_filepaths, ids):   
    print('Working on',edf_filepath)
    
    #%%Get stage info
    epoch_stages, epochoffset, starttime = scorefiles.extract_epochstages_from_scorefile(scorefile_filepath, 
                                                                                         study_settings['stage_map'])
    epoch_stages = scorefiles.score_wake_as_waso_wbso_wase(epoch_stages)

    #get the start and end of where we want to extract spindles from (lights off->lights on)
    lights_off_secs, lights_on_secs, \
    _, epoch_stages = sleep_architecture.lights_on_off_and_sleep_latency(epoch_stages,
                                                                         epoch_sync_offset_seconds=epochoffset)
    
    edf = mne.io.read_raw_edf(edf_filepath)
    all_eeg_chans = [v for v in study_settings['known_eeg_chans'].keys() if v in edf.ch_names]
    
    
    features_df, power_df, feature_averages_df, power_averages_df, epochs_with_artifacts = \
        process_sleep_record.extract_eeg_variables(edf_filepath,
                          epoch_stages,
                          epochoffset_secs=lights_off_secs,
                          end_offset=lights_on_secs,
                          do_artifacting=do_artifacting,
                          do_spindles=do_spindles,
                          do_slow_osc=do_slow_osc,
                          do_band_power=do_band_power,
                          artifacting_channels=all_eeg_chans,
                          spindle_channels=all_eeg_chans,
                          slow_osc_channels=all_eeg_chans,
                          band_power_channels=all_eeg_chans,
                          do_quartiles=nighttime_split_method=='quartile',
                          do_overlap=do_overlap,
                          timeit=True)
    
    features_df['subjectid'] = id_
    feature_averages_df['subjectid'] = id_
    features_df.to_csv(save_base_path+'sleep_features_subjectid'+str(id_)+'.csv')
    features_averages_cont.append(feature_averages_df)
    features_df_cont.append(features_df)
    if do_band_power:
        power_df['subjectid'] = id_
        power_averages_df['subjectid'] = id_
        band_power_averages_cont.append(power_averages_df)

    print(datetime.datetime.now(), '\tSleep Features extraction finished')
    

Working on C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
0:00:00 	Feature extraction started
Spindle Extraction starting for C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
Detected 5801 spindles on C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
Spindle extraction took 0:01:29.297142
Bundeling extraction took 0:00:00.034998
Slow Osc Extraction starting for C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
Detected 43528 slow osc for C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
Slow Osc extraction took 0:00:13.962006
	Features took 0:01:45.498146
2019-09-04 16:10:09.943361 	Sleep Features extraction finished


In [13]:
#Combine all subs into a single dataframe and save to csv
if do_band_power:
    all_band_info = pd.concat(band_power_averages_cont, axis=0)
    all_band_info.to_csv(save_base_path + 'band_power_averages_data.csv', index=False)

all_feature_info = pd.concat(features_averages_cont, axis=0)
all_feature_info.to_csv(save_base_path + 'sleep_feature_averages_data.csv', index=False)

all_feature_df= pd.concat(features_df_cont, axis=0)
all_feature_df.to_csv(save_base_path + 'all_individual_sleep_features.csv', index=False)

TypeError: cannot concatenate object of type "<class 'list'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [22]:
features_df

,duration,chan,onset,freq_peak,peak_time,peak_uV,description,stage,stage_idx,trough_uV,zero_time,trough_time,subjectid
0,0.810,O2-M1,692.660,7.407407,0.420,8.529264e+05,spindle,n1,23,NaN,NaN,NaN,1
1,1.120,O1-M2,692.825,8.928571,0.605,6.268155e+05,spindle,n1,23,NaN,NaN,NaN,1
2,0.665,F4-M1,694.135,19.548872,0.340,3.249367e+05,spindle,n1,23,NaN,NaN,NaN,1
3,0.605,O1-M2,694.145,8.264463,0.305,5.637375e+05,spindle,n1,23,NaN,NaN,NaN,1
4,0.805,O1-M2,697.960,8.695652,0.445,7.139177e+05,spindle,n1,23,NaN,NaN,NaN,1
5,0.320,F4-M1,701.400,9.375000,0.195,1.957708e+05,spindle,n1,23,NaN,NaN,NaN,1
6,0.660,O1-M2,702.135,10.606061,0.265,3.377513e+05,spindle,n1,23,NaN,NaN,NaN,1
7,0.945,O1-M2,703.150,13.756614,0.420,3.333749e+05,spindle,n1,23,NaN,NaN,NaN,1
8,0.780,O2-M1,705.545,14.102564,0.445,1.288855e+06,spindle,n1,23,NaN,NaN,NaN,1
9,0.435,O1-M2,707.260,9.195402,0.100,2.320979e+05,spindle,n1,23,NaN,NaN,NaN,1


In [ ]:
all_feature_info